In [1]:
import os
os.chdir('..')

from simulator.noisy_simulator import NoisySimulator
from data_objects.backend import LinearBackend
from simulator.readout_error_model import ReadoutErrorModel
from optimizations.readout_mitigation.fem import Mitigator
from optimizations.readout_mitigation.fem import IterativeSamplingProtocol, EnumeratedProtocol
from data_objects.algorithms import get_algs, ibu_response_matrix
from qiskit.quantum_info.analysis import hellinger_fidelity
from optimizations.readout_mitigation.fem.tools import npformat_to_statuscnt,statuscnt_to_npformat
from time import time
import pickle
import matplotlib.pyplot as plt
import logging
logging.getLogger('qiskit').setLevel(logging.ERROR)

# Readout Calibration Of JanusQ-FEM on Real-world Quantum Device 
**Author:** Hanyu Zhang \& Siwei Tan

**Date:** 9/4/2024

Based on paper "[QuFEM: Fast and Accurate Quantum Readout Calibration Using the Finite Element Method][1]"

[1]: https://scholar.google.com/scholar_url?url=https://dl.acm.org/doi/abs/10.1145/3613424.3614274%3Fcasa_token%3DffjIB1hQ4ZwAAAAA:8MajDLrDOC74WoeMf7r7AoQ-koxCa4E1TNqQg3GSDz03xUX6XdE3toNTM-YdM_e4rKEusMceJ6BGJg&hl=zh-CN&sa=T&oi=gsb&ct=res&cd=0&d=11146218754516883150&ei=42YSZpPlFL6s6rQPtt6x6Ac&scisig=AFWwaeYaiu2hyx8HUJ_7Buf9Mwom

We run a GHZ circuit on the QuaFu [] quantum computer and load its data. Then, we compare its fidelity with the ideal results without noise.

TODO: 加上引用
TODO: 热力图

We fist load the dataset for the notebook.

In [ ]:
# TODO: 用相对路径

benchmark_circuits_and_results = pickle.load(file=open('/home/zhanghanyu/JanusQ/dataset/protocol_16.pickle','rb'))

ghz_output = pickle.load(file=open('/home/zhanghanyu/JanusQ/dataset/ghz_error','rb'))
# TODO: 生成一个数组包含了5比特到131比特的GHZ输出 （还包含读取的比特）

ibu_matrices = pickle.load(file=open('/home/zhanghanyu/JanusQ/dataset/matrices_16.pickle','rb'))


## JanusQ-CT

In [ ]:
# TODO: construct mitigator

# TODO: 构建一个131比特的mitigator
# 用这个校准不同比特数的GHZ结果

# t_qufem_new = []
# qufem_fid = []

# miti = []
# for i in range(len(ghz_output)):
#     n_qubits = 6+i*2
#     benchmark_circuits = downsample(benchmark,[i for i in range(n_qubits)])

#     outout_ideal = {'1'*n_qubits:samples*0.5,'0'*n_qubits:samples*0.5}

#     bstrs = []
#     for key in benchmark_circuits.keys():
#         bstrs.append(np.array(list(key), dtype=np.int8))

#     protocol_statuscnts = []
#     for value in benchmark_circuits.values():
#         protocol_statuscnts.append(value)

    
#     t_qufem_1_new = time()
#     output_fem = mitigator[i].mitigate(ghz_output[i])
#     t_qufem_2_new = time()
#     output_fem = npformat_to_statuscnt(output_fem)

#     t_qufem_new.append(t_qufem_2_new-t_qufem_1_new)
#     qufem_fid.append(hellinger_fidelity(outout_ideal,output_fem))


In [ ]:
# Calibrate GHZ

## Comparision to IBU

We have collected 5-qubit to 131-qubit GHZ circuit output on a 136-qubit quantum device. We compare the calibration accuracy and overhead of JanusQ-CT to IBU [].

TODO: 加上引用

In [ ]:
# TODO: 生成一个数组包含了5比特到131比特的GHZ输出 （还包含读取的比特）

# collect the GHZ outputs
# samples = 10000
# un_fid = []
# for i in range(len(ghz_output)):
#     n_qubits = 6+i*2
#     ghz_noisy = ghz_output[i]
#     outout_ideal = {'1'*n_qubits:samples*0.5,'0'*n_qubits:samples*0.5}
#     un_fid.append(hellinger_fidelity(ghz_noisy,outout_ideal))

In [ ]:
# construct the IBU mitigator and apply calibration
from baselines.readout_calibration.IBU.src.IBU import IBU
from baselines.readout_calibration.IBU.utils.qc_utils import *
from baselines.readout_calibration.IBU.utils.data_utils import *

t_ibu_new = []
ibu_fid = []
for i in range(len(ghz_output)):
    n_qubits = 6+i*2
    params = {
        "exp_name": "ghz",
        "method": "reduced",  
        "library": "jax",  
        "num_qubits": n_qubits,
        "max_iters": 1000,
        "tol": 1e-5,
        "use_log": False,  
        "verbose": True,
        "init": "unif",  
        "smoothing": 1e-8,
        "ham_dist": 3
    }

    matrices_ibu = ibu_matrices[:n_qubits]
    outout_ideal = {'1'*n_qubits:samples*0.5,'0'*n_qubits:samples*0.5}


    ibu_new = IBU(matrices_ibu, params)
    ghz_noisy = ghz_output[i]
    ibu_new.set_obs(dict(ghz_noisy))
    t_ibu_new_1 = time()
    ibu_new.initialize_guess()
    t_sol_new, max_iters_new, tracker_new = ibu_new.train(params["max_iters"], tol=params["tol"], soln=outout_ideal)
    t_ibu_new_2 = time()
    outout_ibu_new = ibu_new.guess_as_dict()

    t_ibu_new.append(t_ibu_new_2-t_ibu_new_1)
    ibu_fid.append(hellinger_fidelity(outout_ideal,outout_ibu_new).item())

In [ ]:
# compare the fidelity improvement

x = [6+i*2 for i in range(len(ghz_output))]

plt.plot(x, un_fid, label='RAW', marker='^', linestyle='-')
plt.plot(x, ibu_fid, label='IBU', marker='^', linestyle='-')
plt.plot(x, qufem_fid, label='QuFEM', marker='^', linestyle='-')
plt.xticks(x) 
plt.title('Fidelity Comparison')
plt.xlabel('#qubits')
plt.ylabel('fidelity')

plt.legend()

plt.show()


In [ ]:
# compare the time cost

x = [6+i*2 for i in range(len(ghz_output))]

plt.plot(x, t_ibu_new, label='IBU', marker='^', linestyle='-')
plt.plot(x, t_qufem_new, label='QuFEM', marker='^', linestyle='-')
plt.xticks(x) 
plt.title('Time Comparison')
plt.xlabel('#qubits')
plt.ylabel('time')

plt.legend()

plt.show()